In [1]:
# 1. (推荐) 设置 iPython 自动重载
#    这能让您在修改 .py 文件后，Notebook 能自动加载最新版本
#    无需重启内核
%load_ext autoreload
%autoreload 2

# 2. (关键) 更改当前工作目录 (Change Directory)
#    '..' 代表上一级目录
#    执行后，您的 Notebook 将会从"项目根目录"运行
%cd ..

import arcpy

# 导入您的 SoilSampler 类
# (现在可以无缝工作了)
from script.Function import SoilSampler

e:\Soil\SamplingPoint


| 应用场景 | 推荐采样密度<br>(点/公顷) | 对应的网格大小<br>(近似值) | 适用目标 | 科学依据与参考标准 |
| :--- | :--- | :--- | :--- | :--- |
| **A. 精准农业研究**<br>(High Precision) | **1.0 ~ 3.0** | $60m \times 60m$<br>至<br>$100m \times 100m$ | **制作高精度处方图 (VRA)**<br>科研级验证、微小变异捕捉。 | **网格采样法 (Grid Sampling)**<br>国际精准农业通用标准。通常要求至少达到 1点/公顷 以捕捉土壤养分的空间结构。对于高变异地块，需加密至 0.4公顷/点 (约2.5点/公顷)。 |
| **B. 分区管理采样**<br>(Zone Management)<br>⚡ **(本项目推荐)** | **0.5 ~ 1.5** | $80m \times 80m$<br>至<br>$140m \times 140m$ | **UAV 能谱/遥感指导采样**<br>利用影像已知的变异信息，在“高/中/低”值区针对性布点。 | **辅助变量指导采样 (Targeted Sampling)**<br>相比盲目的网格法，利用遥感作为先验知识可减少 30%-50% 的采样量而不损失精度。学术界推荐在地块尺度下，依据变异系数 (CV) 灵活调整。 |
| **C. 地统计学建模**<br>(Geostatistics) | **> 2.0**<br>(或总数 >30) | $< 70m \times 70m$ | **构建克里金变异函数**<br>为了获得稳定的半变异函数曲线，必须有足够的点对。 | **地统计学原理 (Kerry & Oliver)**<br>构建可靠的半变异函数通常要求单个研究区域内的样本总数不低于 30-50 个，无论面积大小。 |
| **D. 常规土壤调查**<br>(General Survey) | **0.03 ~ 0.2** | $> 200m \times 200m$ | **测土配方施肥/普查**<br>了解区域平均水平，忽略局部细节。 | **中国农业行业标准 (NY/T 2911-2016)**<br>《测土配方施肥技术规范》规定：<br>• 平原区：100~500亩 (约6.7~33.3公顷) 采一个样。<br>• 丘陵区：30~80亩 (约2~5.3公顷) 采一个样。 |

In [2]:
# 工作空间 GDB
WORKSPACE_GDB = r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb"

# 批量配置
MY_BATCH_CONFIG = [
    {
        "id": "Region_A",
        "points": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Peaks\Region_A_peaks_point",
        "farmland": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Region\Region_A"
    },
    {
        "id": "Region_B",
        "points": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Peaks\Region_B_peaks_point",
        "farmland": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Region\Region_B"
    },
    {
        "id": "Region_C",
        "points": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Peaks\Region_C_peaks_point",
        "farmland": r"E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb\Region\Region_C"
    },
]

# 投影坐标系 (UTM)
UTM_SR = "WGS 1984 UTM Zone 50N"

# 样式模板路径 (可选，如没有可设为 None)
STYLE_TEMPLATE = r"E:\Documents\ArcGIS\Projects\SamplingPoint\style\sample_points_classified_style.lyrx"

# 这里想要分析的所有字段！
# 可以是一个，也可以是多个
TARGET_FIELDS = ["total", "K40_peak", "Bi214_peak", "Tl208_peak"]
# TARGET_FIELDS = ["Tl208_peak"] # 如果只想跑一个，这样写


# ----------------------------------------------------------
# 智能参数配置
# ----------------------------------------------------------
# 模式: "auto" (根据农田面积自动计算数量和间距)
SAMPLING_MODE = "auto"

# 密度: 每公顷采 DENSITY_PER_HA 个点 
# 1公顷 = 100m x 100m ≈ 15市亩
DENSITY_PER_HA = 0.5

# 保底: 无论地多小，至少采 5 个点
MIN_SAMPLE_COUNT = 5

# 比例: 重点关注高/低异常值 (Hot/Cold), 中间值 (Normal) 少采
RATIO_CONFIG = {
    "hotPoint": 0.4,    # 40%
    "coldPoint": 0.4,   # 40%
    "normalPoint": 0.2  # 20%
}

# ==========================================================
# 执行主程序
# ==========================================================
# 检查 Spatial Analyst 许可
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    
    try:
        print("正在初始化 SoilSampler")
        print("-" * 60)
        
        # 实例化工具类
        sampler = SoilSampler(
            workspace=WORKSPACE_GDB,
            batch_config=MY_BATCH_CONFIG,
            output_utm_sr=UTM_SR,
            
            # --- 多要素字段 ---
            value_fields=TARGET_FIELDS,
            time_field="collection_time", # 原始数据中用于生成航线的时间字段名
            
            # --- 智能采样参数 ---
            mode=SAMPLING_MODE,
            sampling_density=DENSITY_PER_HA,
            min_sample_count=MIN_SAMPLE_COUNT,
            ratio_config=RATIO_CONFIG,
            
            # --- 算法参数 ---
            kriging_model_str="SPHERICAL",
            
            # [关键设置] 自动计算步长
            # 设为 "AUTO": 脚本会调用平均最近邻分析，自动算出最适合该数据的步长(如 3.35)
            # 设为 数字 : 强制使用固定步长
            kriging_range="AUTO",  
            
            line_point_distance="10 Meters",
            
            # --- 系统参数 ---
            symbology_template_lyrx=STYLE_TEMPLATE,
            overwrite=True
        )
        
        # 运行批量处理
        sampler.run_batch()
        
    except Exception as e:
        print(f"\n!! 程序运行遇到错误: {e}")
        # 打印详细报错栈，方便调试
        import traceback
        traceback.print_exc()
        
    finally:
        # 归还许可，释放资源
        arcpy.CheckInExtension("Spatial")
        print("-" * 60)
        print("处理结束，许可已归还。")
else:
    print("错误: 无法获取 ArcGIS Spatial Analyst 扩展许可。")

正在初始化 SoilSampler
------------------------------------------------------------
工作空间(GDB): E:\Documents\ArcGIS\Projects\SamplingPoint\SamplingPoint.gdb
栅格输出(Grid): E:\Documents\ArcGIS\Projects\SamplingPoint\Grid
开始批量处理 (v13.3 日志增强版)...

=== 正在处理区域: Region_A ===
  [自动分析] 计算点间距...
  [自动分析] 最佳步长: 78.1469 米

  >>> 正在处理要素: [total] ...
      -> 插值成功: Kriging_Region_A_total.tif
      [智能参数] 农田:38.11ha | 计划:20点 | 目标间距(网格):110.4m
      -> 网格法未收敛，切换至[距离约束随机]模式...
      -> 正在寻找满足 55.2米 间距的组合...
      -> 成功找到方案! (尝试第 1 次)
      4. 导出采样点...
      -> 样式已应用: Sample_Region_A_total.lyrx
      -> 表格已导出: Coords_Region_A_total.csv

  >>> 正在处理要素: [K40_peak] ...
      -> 插值成功: Kriging_Region_A_K40_peak.tif
      [智能参数] 农田:38.11ha | 计划:20点 | 目标间距(网格):110.4m
      4. 导出采样点...
      -> 样式已应用: Sample_Region_A_K40_peak.lyrx
      -> 表格已导出: Coords_Region_A_K40_peak.csv

  >>> 正在处理要素: [Bi214_peak] ...
      -> 插值成功: Kriging_Region_A_Bi214_peak.tif
      [智能参数] 农田:38.11ha | 计划:20点 | 目标间距(网格):110.4m
      -> 网格法未收敛，切